### Gmarket 베스트 상품 데이터 수집

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
url = "http://corners.gmarket.co.kr/Bestsellers"
response = requests.get(url)
dom = BeautifulSoup(response.content, "html.parser")
items = dom.select("#gBestWrap > div > div:nth-child(6) > div:nth-child(3) > ul > li")
len(items)

200

In [10]:
datas = []
for item in items:
    data = {
        "link": item.select_one(".itemname").get("href"),
        "title": item.select_one(".itemname").text,
        "o-price": item.select_one(".o-price").text.strip().split("원")[0],
        "s-price": item.select_one(".s-price").text.strip().split("원")[0],
    }
    datas.append(data)
    
items_df = pd.DataFrame(datas)
items_df.tail()

,link,title,o-price,s-price
195,http://item.gmarket.co.kr/Item?goodscode=14792...,[나무야나무야]나무야 도톰한 3겹데코 30m 30롤 3팩/화장지/휴지,,"31,900"
196,http://item.gmarket.co.kr/Item?goodscode=17371...,[제스파]바디볼트 전신 안마기 진동 무선 마사지건 ZP2420,"119,000","116,620"
197,http://item.gmarket.co.kr/Item?goodscode=15089...,[매일유업]매일 상하 유기농 어린이치즈 3단계 110매/치즈,"29,000","26,900"
198,http://item.gmarket.co.kr/Item?goodscode=13073...,[크레모아]LED 캠핑랜턴 울트라 미니 블랙에디션 (CLC-400BK),,"59,000"
199,http://item.gmarket.co.kr/Item?goodscode=17952...,20SS 할리마리 플라워 풍기 인견 원피스2종,"59,800","52,820"


In [11]:
idx = items_df["o-price"] == ""

In [12]:
items_df.loc[idx, "o-price"] = items_df["s-price"]

In [13]:
items_df["code"] = items_df["link"].apply(lambda link: link.split("goodscode=")[1].split("&")[0])
items_df = items_df[["code", "title", "o-price", "s-price", "link"]]
items_df.tail()

,code,title,o-price,s-price,link
195,1479209063,[나무야나무야]나무야 도톰한 3겹데코 30m 30롤 3팩/화장지/휴지,"31,900","31,900",http://item.gmarket.co.kr/Item?goodscode=14792...
196,1737189878,[제스파]바디볼트 전신 안마기 진동 무선 마사지건 ZP2420,"119,000","116,620",http://item.gmarket.co.kr/Item?goodscode=17371...
197,1508911526,[매일유업]매일 상하 유기농 어린이치즈 3단계 110매/치즈,"29,000","26,900",http://item.gmarket.co.kr/Item?goodscode=15089...
198,1307322003,[크레모아]LED 캠핑랜턴 울트라 미니 블랙에디션 (CLC-400BK),"59,000","59,000",http://item.gmarket.co.kr/Item?goodscode=13073...
199,1795266739,20SS 할리마리 플라워 풍기 인견 원피스2종,"59,800","52,820",http://item.gmarket.co.kr/Item?goodscode=17952...


#### 이미지 파일 저장

In [14]:
import os

In [15]:
# 디렉토리 만들기
if not os.path.exists("datas"):
    os.makedirs("datas")

In [16]:
def save_file(title, link):
    response = requests.get(link)
    with open('datas/{}.png'.format(title), 'wb') as f:
        f.write(response.content)

In [17]:
for idx, data in items_df.iterrows():
    img_link = "http://gdimg.gmarket.co.kr/{}/still/300".format(data["code"])
    idx = "0" * (3 - len(str(idx))) + str(idx)   
    save_file("{}_{}".format(idx, data["code"]), img_link)

In [18]:
!rm -rf datas

'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
